# CISC3024 Machine Learning Final Project
- Title: Wound Detection
- Groupmates: Huang Yanzhen DC126732, Yang Zhihan DC127992

In [1]:
# Basics
import os
import copy
import time
from itertools import product
from typing import List, Callable, Any, Union

# Pre-processing
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from sklearn.metrics import mean_squared_error
import pandas as pd

# Model Training
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

### Data Retrieval

In [2]:
def get_labels(data_type: str) -> np.ndarray:
    """
    Get ground truth labels from .csv file.
    :param data_type: Type of data: Training or Testing.
    """
    df = pd.read_csv(f"./Wound/{data_type}/myData.csv", delimiter=";")
    return df.to_numpy()

In [3]:
def get_images(data_type: str,
               image_names: np.ndarray,
               augmentation: Union[Callable[[np.ndarray, Any], np.ndarray], None] = None,
               flatten=True,
               **kwargs) -> np.ndarray:
    """
    Get the images from directory.
    :param data_type: Type of data: Training or Testing.
    :param image_names: Names of images from ground truth.
    :param augmentation: Augmentation function.
    :param flatten: Whether to flatten the images.
    :param kwargs: Other arguments to pass to augmentation function.
    """
    images = []
    for i_name in image_names:
        img = Image.open(os.path.join(f"./Wound/{data_type}/", i_name))
        img = img.resize((32, 32), Image.BICUBIC)
        img = np.array(img)
        if augmentation:
            img = augmentation(img, **kwargs)
        images.append(img.flatten() if flatten else img)

    images = np.array(images)

    return images

### Image Augmentation

In [4]:
def add_black_edge(img: np.array, w: int = 4) -> np.array:
    """
    Image augmentation. Add an inner black edge to an image.
    :param img: Image to be processed.
    :param w: Width of the edge.
    """
    if w > min(img.shape[0:2]) // 2:
        raise ValueError("Width of the edge must be smaller than half of the shorter side of an image.")

    new_img = np.zeros_like(img)
    new_img[w:-w, w:-w, :] = img[w:-w, w:-w, :]
    return new_img

In [5]:
def stretch(img: np.ndarray, f: List[float]) -> np.ndarray:
    """
    Image augmentation. Stretch an image on the width and height side.
    :param img: Image to be augmented.
    :param f: Factor tuple. Width and Height.
    """
    fw, fh = f
    if fw < 1 or fh < 1:
        raise ValueError("Width and height factors should be greater than or equal to 1.")

    # New widths
    new_width = int(img.shape[1] * fw)
    new_height = int(img.shape[0] * fh)

    # Adjust image
    img_pil = Image.fromarray(img)
    img_resized = img_pil.resize((new_width, new_height), Image.BICUBIC)

    # Crop regions
    # Keep 32x32 size
    left = (new_width - 32) // 2
    top = (new_height - 32) // 2
    right = left + 32
    bottom = top + 32

    # Crop image
    img_cropped = img_resized.crop((left, top, right, bottom))

    # Convert to numpy array
    img_stretched = np.array(img_cropped)

    return img_stretched

### Train Model
One trainig of model would result in the following structure:
```python
{
    "x": {
        "Best MSE": smallest_mse,
        "Best Fold": best_fold_idx,
        "Avg MSE": avg_mse,
        "model": ModelInstance,
    },
    "y": {
        "Best MSE": smallest_mse,
        "Best Fold": best_fold_idx,
        "Avg MSE": avg_mse,
        "model": ModelInstance,
    },
    "w": {
        "Best MSE": smallest_mse,
        "Best Fold": best_fold_idx,
        "Avg MSE": avg_mse,
        "model": ModelInstance,
    },
    "h": {
        "Best MSE": smallest_mse,
        "Best Fold": best_fold_idx,
        "Avg MSE": avg_mse,
        "model": ModelInstance,
    },
}
```
This is named as an "experiment object".

In [43]:
def train(ModelInstance, X, Y, desc: str = "DESC", n_fold: int = 3, save: bool = False):
    """
    Train the model. Output would be of shape:
    :param ModelInstance: Instance of a model class.
    :param desc: Description of the saved file.
    :param n_fold: Number of folds.
    :param save: Whether to save the experiment object.
    """
    model_name = ModelInstance.__class__.__name__
    semantic_y = ["File Name", "x", "y", "w", "h"]

    # Print Model configurations
    print(f"Training model {model_name}. Description: {desc}\nStarted at: {time.time()}")
    # Predict all for x, y, w, h
    exp = {}
    for i in range(1, Y.shape[1]):
        # Totally 4 labels to predict.
        # Select one of them.
        y = Y[:, i]

        # Split original data into 3 parts
        # to perform cross-validation
        kf = KFold(n_splits=n_fold, shuffle=True, random_state=1919810)
        splits = kf.split(X)

        # Record MSE of each fold.
        # Keep the model with the smallest MSE
        mse_scores = []
        cur_best_model = None
        cur_smallest_MSE = np.inf
        for train_index, val_index in splits:
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]

            model = copy.deepcopy(ModelInstance)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_val)
            mse = mean_squared_error(y_val, y_pred)
            mse_scores.append(mse)

            if cur_smallest_MSE > mse_scores[-1]:
                cur_best_model = copy.deepcopy(model)

        exp[semantic_y[i]] = {
            "Best MSE": cur_smallest_MSE,
            "Best Fold": np.argmin(mse_scores),
            "Avg MSE": np.mean(mse_scores),
            "model": copy.deepcopy(cur_best_model)
        }
        del cur_best_model

        print(f"{semantic_y[i]} - Avg MSE={np.mean(mse_scores):.4f}, "
              f"Best MSE={np.min(mse_scores):.4f} at index {np.argmin(mse_scores)}")

    # Save models
    print(f"Ended at {time.time()}\n\n")
    if save:
        time_str = str(time.time()).replace(".", "")
        pickle.dump(exp, open(f"./save_models/{model_name}_{desc}_{time_str}.sav", "wb"))
    return exp

In [44]:
def grid_search(ModelClass, hyper_params, hyper_param_names, kwarg_names, **kwargs):
    """
    Perform grid search for the given model class and hyperparameters.
    :param ModelClass: The class to instantiate the model.
    :param hyper_params: The product of two lists of candidate hyperparameters.
    :hyper_param_names: The display name of two hyperparameters.
    :kwarg_names: Names of keyword arguments to be passed into the model.
    :returns: A list of dictionaries containing hyperparameters and experiment objects.
    """
    param_exps = []
    
    n1, n2 = hyper_param_names
    kw1, kw2 = kwarg_names
    
    model_name = ModelClass.__name__
    
    # Data Augmentation
    # Change some useless information
    Y_ori = get_labels(data_type="Training")
    X_ori = get_images(data_type="Training", image_names=Y_ori[:, 0])

    # Add black edge
    Y_be = get_labels(data_type="Training")
    X_be = get_images(data_type="Training", image_names=Y_be[:, 0], augmentation=add_black_edge, w=4)

    # Stretch height
    Y_sh = get_labels(data_type="Training")
    X_sh = get_images(data_type="Training", image_names=Y_sh[:, 0], augmentation=stretch, f=[1.0, 1.05])
    Y_sh[:, 4] *= 1.05

    # Stretch Width
    Y_sw = get_labels(data_type="Training")
    X_sw = get_images(data_type="Training", image_names=Y_sw[:, 0], augmentation=stretch, f=[1.05, 1.0])
    Y_sw[:, 3] *= 1.05
    
    X = np.concatenate((X_ori, X_be, X_sh, X_sw))
    Y = np.concatenate((Y_ori, Y_be, Y_sh, Y_sw))
    
    for param1, param2 in hyper_params:
        # Dynamically add the hyperparameters to kwargs
        model_kwargs = {
            kw1: param1,
            kw2: param2
        }
        
        model_kwargs.update(kwargs)
        
        model_instance = ModelClass(**model_kwargs)
        exp = train(model_instance, X, Y, desc=f"{n1}-{param1}--{n2}-{param2}", n_fold=3, save=False)
        param_exps.append({
            n1: param1,
            n2: param2,
            "exp": exp
        })

    time_str = str(time.time()).replace(".", "")
    pickle.dump(param_exps, open(f"./save_models/{model_name}_{n1}-{n2}_{time_str}.sav", "wb"))
    return param_exps

In [45]:
def test(exp_list, Y_test, X_test):
    # Y_test = get_labels(data_type="Test")
    # X_test = get_images(data_type="Test", image_names=Y_test[:,0])
    results = []
    for i, exp in enumerate(exp_list):
        param_name1, param_name2, _ = exp.keys()
        param1, param2, models = list(exp.values())
        
        model_x, model_y, model_w, model_h = (models["x"]["model"], 
                                              models["y"]["model"], 
                                              models["w"]["model"],
                                              models["h"]["model"])
        
        y_x, y_y, y_w, y_h = Y_test[:, 1], Y_test[:, 2], Y_test[:, 3], Y_test[:, 4]
        
        y_pred_x, y_pred_y, y_pred_w, y_pred_h = (model_x.predict(X_test), 
                                                  model_y.predict(X_test), 
                                                  model_w.predict(X_test), 
                                                  model_h.predict(X_test))
        
        mse_x, mse_y, mse_w, mse_h = (mean_squared_error(y_x, y_pred_x), 
                                      mean_squared_error(y_y, y_pred_y), 
                                      mean_squared_error(y_w, y_pred_w), 
                                      mean_squared_error(y_h, y_pred_h))
        
        avg_mse = np.mean([mse_x, mse_y, mse_w, mse_h])
        
        results.append({
            param_name1: param1,
            param_name2: param2,
            "avg_mse": avg_mse
        })
    return results

### Grid Search: RandomForestRegressor
Hyper parameters for Random Forest Regressor:
- n_estimators: Number of estimators.
- bootstrap: Bootstrap or not.
- max_depth: Maximum Depth of the tree.
- min_samples_split: Minimum sample number that allows a leaf to be split again.
- min_samples_leaf: Minimum sample number a leaf requires.

In [8]:
rfr_nest = [10, 20, 30, 40, 50]
rfr_maxd = [11, 13, 15, 17, 19]
rfr_mins = [2, 4, 6, 8, 10]
rfr_minl = [6, 8, 10, 12, 14]
rfr_grid0 = product(rfr_nest, rfr_maxd)
rfr_grid1 = product(rfr_mins, rfr_minl)

### Grid Search 0: Num Estimators + Max Deapth

In [9]:
rfr_grid0_exp = grid_search(ModelClass=RandomForestRegressor, 
                            hyper_params=rfr_grid0, 
                            hyper_param_names=["nest", "maxd"], 
                            kwarg_names=["n_estimators", "max_depth"])

Training model RandomForestRegressor. Description: nest-10--maxd-11
Started at: 1732799242.455379
x - Avg MSE=265.0281, Best MSE=226.6704 at index 1
y - Avg MSE=319.6748, Best MSE=206.1377 at index 1
w - Avg MSE=981.5292, Best MSE=914.7635 at index 0
h - Avg MSE=1024.7640, Best MSE=774.9044 at index 0
Ended at 1732799277.573622

Training model RandomForestRegressor. Description: nest-10--maxd-13
Started at: 1732799303.3531008
x - Avg MSE=303.6308, Best MSE=264.3288 at index 2
y - Avg MSE=330.7782, Best MSE=254.7151 at index 1
w - Avg MSE=1046.2773, Best MSE=935.1649 at index 0
h - Avg MSE=1015.2763, Best MSE=801.2331 at index 1
Ended at 1732799339.3567605

Training model RandomForestRegressor. Description: nest-10--maxd-15
Started at: 1732799364.6094503
x - Avg MSE=300.3605, Best MSE=246.8027 at index 2
y - Avg MSE=316.6453, Best MSE=246.9491 at index 1
w - Avg MSE=1039.2064, Best MSE=971.3801 at index 0
h - Avg MSE=1130.1478, Best MSE=980.0463 at index 1
Ended at 1732799401.170166

Tr

In [26]:
with open("./save_models/RandomForestRegressor_nest-maxd_17328034630916922.sav", "rb") as rfr_grid0_exp_f:
    rfr_grid0_exp_loaded = pickle.load(rfr_grid0_exp_f)

In [38]:
Y_test = get_labels(data_type="Test")
X_test = get_images(data_type="Test", image_names=Y_test[:,0])
rfr_grid0_results = test(exp_list=rfr_grid0_exp_loaded, Y_test=Y_test, X_test=X_test)

In [39]:
rfr_grid0_results

[{'nest': 10, 'maxd': 11, 'avg_mse': np.float64(1121.6735900260442)},
 {'nest': 10, 'maxd': 13, 'avg_mse': np.float64(1207.1127779833778)},
 {'nest': 10, 'maxd': 15, 'avg_mse': np.float64(1397.0920984884651)},
 {'nest': 10, 'maxd': 17, 'avg_mse': np.float64(1207.84312921875)},
 {'nest': 10, 'maxd': 19, 'avg_mse': np.float64(1337.546615156251)},
 {'nest': 20, 'maxd': 11, 'avg_mse': np.float64(1120.27080063905)},
 {'nest': 20, 'maxd': 13, 'avg_mse': np.float64(1222.522943146033)},
 {'nest': 20, 'maxd': 15, 'avg_mse': np.float64(1164.6933728044157)},
 {'nest': 20, 'maxd': 17, 'avg_mse': np.float64(1100.4649255305203)},
 {'nest': 20, 'maxd': 19, 'avg_mse': np.float64(1094.5792512109372)},
 {'nest': 30, 'maxd': 11, 'avg_mse': np.float64(1061.189627912441)},
 {'nest': 30, 'maxd': 13, 'avg_mse': np.float64(1042.1595238991367)},
 {'nest': 30, 'maxd': 15, 'avg_mse': np.float64(1115.1385660510634)},
 {'nest': 30, 'maxd': 17, 'avg_mse': np.float64(1055.0009023263888)},
 {'nest': 30, 'maxd': 19, '

In [40]:
rfr_grid0_best = min(rfr_grid0_results, key=lambda x: x["avg_mse"])
rfr_grid0_best

{'nest': 50, 'maxd': 17, 'avg_mse': np.float64(1036.6432180931386)}

#### Grid Search 1: Min Samples Split, Min Samples Leaf

In [46]:
rfr_grid1_exp = grid_search(ModelClass=RandomForestRegressor, 
                            hyper_params=rfr_grid1, 
                            hyper_param_names=["mins", "minl"], 
                            kwarg_names=["min_samples_split", "min_samples_leaf"],
                            n_estimators=50,    # TODO: Subject to change
                            max_depth=17)

Training model RandomForestRegressor. Description: mins-2--minl-14
Started at: 1732818348.657749
x - Avg MSE=402.2252, Best MSE=363.8324 at index 1
y - Avg MSE=438.6546, Best MSE=365.7809 at index 1
w - Avg MSE=1442.0456, Best MSE=1202.3531 at index 0
h - Avg MSE=1340.2832, Best MSE=1192.4955 at index 0
Ended at 1732818430.85319


Training model RandomForestRegressor. Description: mins-4--minl-6
Started at: 1732818430.85319
x - Avg MSE=317.2188, Best MSE=296.8959 at index 1
y - Avg MSE=329.0119, Best MSE=247.3725 at index 1
w - Avg MSE=1046.5065, Best MSE=946.3808 at index 0
h - Avg MSE=1028.4777, Best MSE=894.0832 at index 0
Ended at 1732818549.377478


Training model RandomForestRegressor. Description: mins-4--minl-8
Started at: 1732818549.377478
x - Avg MSE=343.9908, Best MSE=303.2356 at index 1
y - Avg MSE=363.2314, Best MSE=287.7339 at index 1
w - Avg MSE=1132.5766, Best MSE=1006.1077 at index 0
h - Avg MSE=1018.8811, Best MSE=931.3654 at index 0
Ended at 1732818654.8067505


Trai

In [51]:
with open("./save_models/RandomForestRegressor_mins-minl_1732820399369049.sav", "rb") as rfr_grid1_exp_f:
    rfr_grid1_exp_loaded = pickle.load(rfr_grid1_exp_f)

In [52]:
Y_test = get_labels(data_type="Test")
X_test = get_images(data_type="Test", image_names=Y_test[:,0])
rfr_grid1_results = test(exp_list=rfr_grid1_exp_loaded, Y_test=Y_test, X_test=X_test)

In [53]:
rfr_grid1_results

[{'mins': 2, 'minl': 14, 'avg_mse': np.float64(1057.248578665347)},
 {'mins': 4, 'minl': 6, 'avg_mse': np.float64(1063.385488393176)},
 {'mins': 4, 'minl': 8, 'avg_mse': np.float64(1029.513257691789)},
 {'mins': 4, 'minl': 10, 'avg_mse': np.float64(1008.6793488795022)},
 {'mins': 4, 'minl': 12, 'avg_mse': np.float64(1084.6243709991268)},
 {'mins': 4, 'minl': 14, 'avg_mse': np.float64(1091.1116916882079)},
 {'mins': 6, 'minl': 6, 'avg_mse': np.float64(1043.2612150722418)},
 {'mins': 6, 'minl': 8, 'avg_mse': np.float64(996.6544597092501)},
 {'mins': 6, 'minl': 10, 'avg_mse': np.float64(1013.7592925195522)},
 {'mins': 6, 'minl': 12, 'avg_mse': np.float64(1083.9989861699228)},
 {'mins': 6, 'minl': 14, 'avg_mse': np.float64(1044.7343139650568)},
 {'mins': 8, 'minl': 6, 'avg_mse': np.float64(1017.6530954023252)},
 {'mins': 8, 'minl': 8, 'avg_mse': np.float64(1082.7409997327889)},
 {'mins': 8, 'minl': 10, 'avg_mse': np.float64(1046.5079053489394)},
 {'mins': 8, 'minl': 12, 'avg_mse': np.float

In [55]:
rfr_grid1_best = min(rfr_grid1_results, key=lambda x: x["avg_mse"])
rfr_grid1_best

{'mins': 6, 'minl': 8, 'avg_mse': np.float64(996.6544597092501)}